# What
This is a notebook to show properties that meet certain criteria on a map

# Why
- The filters on realtor sites are bad to nonexistent
- To see price history
- To visualize prices for certain areas/features
# How
## Database
A SQLite Database is used with the setup described below.
Currently ~27K listings take up about 38MB
**Listings**
These listings are scraped from realtor using https://github.com/alexmi256/property-scraper-tools
```
CREATE TABLE Listings (
    Id                             INTEGER PRIMARY KEY
                                           NOT NULL,
    MlsNumber                      INTEGER NOT NULL,
    PublicRemarks                  TEXT    NOT NULL,
    Building_StoriesTotal          TEXT,
    Building_BathroomTotal         INTEGER,
    Building_Bedrooms              INTEGER,
    Building_Type                  TEXT,
    Building_UnitTotal             INTEGER,
    Building_SizeInterior          TEXT,
    Building_SizeExterior          TEXT,
    Property_Type                  TEXT    NOT NULL,
    Property_Address_AddressText   TEXT    NOT NULL,
    Property_Address_Longitude     REAL    NOT NULL,
    Property_Address_Latitude      REAL    NOT NULL,
    Property_Photo_HighResPath     TEXT    NOT NULL,
    Property_ZoningType            TEXT,
    Property_PriceUnformattedValue INTEGER NOT NULL,
    Property_AmmenitiesNearBy      TEXT,
    Property_Parking               TEXT,
    Property_ParkingSpaceTotal     INTEGER,
    Property_OwnershipType         TEXT,
    Land_SizeTotal                 TEXT,
    Land_SizeFrontage              TEXT,
    AlternateURL_DetailsLink       TEXT,
    AlternateURL_VideoLink         TEXT,
    PostalCode                     TEXT    NOT NULL,
    RelativeDetailsURL             TEXT    NOT NULL,
    InsertedDateUTC                TEXT    NOT NULL,
    PriceChangeDateUTC             TEXT,
    ComputedSQFT                   INTEGER,
    ComputedPricePerSQFT           INTEGER,
    ComputedLastUpdated            TEXT,
    ComputedNewBuild               INTEGER
);
```
**PriceHistory**
```
CREATE TABLE PriceHistory (
    MlsNumber INTEGER,
    Price     INTEGER NOT NULL,
    Date      TEXT    NOT NULL,
    FOREIGN KEY (
        MlsNumber
    )
    REFERENCES Listings (MlsNumber),
    UNIQUE (
        MlsNumber,
        Price,
        Date
    )
);
```
Why SQLite?
Because of portability.

# TODO:
- [ ] Fix the DB
- [ ] Add real GeoJSON for metro polylines
- [ ] Clean up and publish the DB merge code
- [ ] Display tables in ipython, ipysheet?
- [ ] A real weighted heatmap

In [1]:
import csv
import json
import logging
import os
import re
import sqlite3
import statistics
import tarfile
from contextlib import closing
from datetime import datetime, timedelta
from pathlib import Path
from urllib.request import urlretrieve

from ipywidgets import HTML, Layout
from ipyleaflet import Map, Marker, basemaps, LayersControl, basemap_to_tiles, AwesomeIcon, Popup, Polyline, FullScreenControl, Heatmap

from geopy import distance
from shapely import Point, Polygon

logging.basicConfig(level=logging.DEBUG)

logger = logging.getLogger(__name__)

In [2]:
class MapViewer:
    def __init__(self, db_file: str, area_of_interest: list[tuple] | None = None):
        """
        Class for displaying visualizations of realty prices
        
        :param db_file:
        """
        self.db_file = db_file
        self.area_of_interest = area_of_interest
        # These should be MLS numbers that I've looked at and never want to see again
        self.blocklist = []
        self.mls_notes = {}
        notes_file = Path("mls_notes.txt")
        if notes_file.exists():
            with open(notes_file) as f:
                lines = f.readlines()
                for line in lines:
                    # lets use, mlsnumber, yes/no, notes
                    data = line.split(",")
                    mls_number = int(data[0].strip())
                    self.mls_notes[mls_number] = {}
                    if len(data) == 1:
                        self.mls_notes[mls_number]["keep"] = False
                    if len(data) > 1:
                        self.mls_notes[mls_number]["keep"] = not (data[1].strip().lower() in ["n", "no", "false", "f"])
                    if len(data) > 2:
                        self.mls_notes[mls_number]["notes"] = data[2].strip()

                self.blocklist.extend([int(x.strip()) for x in f.readlines() if x])

        self.points_of_interest = []
        # https://www.donneesquebec.ca/recherche/dataset/vmtl-stm-traces-des-lignes-de-bus-et-de-metro
        poi_file = Path("stations.geojson")
        if poi_file.exists():
            with open(poi_file) as f:
                poi = json.load(f)
                self.points_of_interest.extend([list(reversed(x["geometry"]["coordinates"])) for x in poi["features"]])

    @staticmethod
    def get_color_for_number_between(number: int | float, minimum: float = 309, maximum: float = 1085) -> str:
        if number < minimum:
            number = minimum
        elif number > maximum:
            number = maximum
        # https://stackoverflow.com/questions/69622670/getting-a-color-range-from-percentage
        ipv = 255 / maximum
        return "#{0:02x}{1:02x}{2:02x}".format(int(ipv * number), 255 - int(ipv * number), 0)

    def get_listings_from_db(
            self,
            min_price: int = 100000,
            max_price: int = 10000000,
            must_have_int_sqft: bool = False,
            must_have_price_change: bool = False,
            no_new_listings: bool = True,
            no_vacant_land: bool = True,
            no_high_rise: bool = True,
            within_area_of_interest: bool = True,
            min_metro_distance_meters: int | None = None,
            min_bedroom: int | None = None,
            min_sqft: int = None,
            max_price_per_sqft: int | None = None,
            last_updated_days_ago: int | None = 14,
            limit: int = -1,
    ) -> list[dict]:
        with closing(sqlite3.connect(self.db_file)) as connection:
            # This helps maintain the row as a dict
            connection.row_factory = sqlite3.Row
            with closing(connection.cursor()) as cursor:
                conditions = []
                if no_vacant_land:
                    conditions.append("Property_ZoningType NOT IN ('Agricultural')")
                if no_high_rise:
                    conditions.append("Building_StoriesTotal IS NULL OR CAST (Building_StoriesTotal AS INTEGER) < 5")
                if no_new_listings:
                    conditions.append("ComputedNewBuild IS NOT TRUE")
                if must_have_int_sqft:
                    conditions.append("Building_SizeInterior IS NOT NULL")
                if must_have_price_change:
                    conditions.append("PriceChangeDateUTC IS NOT NULL")
                if min_bedroom:
                    conditions.append(f"Building_Bedrooms IS NULL OR Building_Bedrooms >= {min_bedroom}")
                if last_updated_days_ago:
                    conditions.append(f"DATE(ComputedLastUpdated) >= DATE('now', '-{last_updated_days_ago} day')")
                if min_sqft:
                    conditions.append(f'ComputedSQFT IS NULL OR ComputedSQFT >= {min_sqft}')
                if max_price_per_sqft:
                    conditions.append(f'ComputedPricePerSQFT IS NULL OR ComputedPricePerSQFT <= {max_price_per_sqft}')
                if limit != -1:
                    conditions.append(f"LIMIT {limit}")

                conditions = [f'({x})' for x in conditions]
                query = f"""
                    SELECT Id,
                           MlsNumber,
                           Property_Address_AddressText,
                           Property_Address_Longitude,
                           Property_Address_Latitude,
                           Property_PriceUnformattedValue,
                           Property_ParkingSpaceTotal,
                           Property_Parking,
                           Property_OwnershipType,
                           Property_Type,
                           Property_Photo_HighResPath,
                           Property_AmmenitiesNearBy,
                           InsertedDateUTC,
                           PriceChangeDateUTC,
                           Building_StoriesTotal,
                           Building_BathroomTotal,
                           Building_Bedrooms,
                           Building_Type,
                           Building_UnitTotal,
                           Building_SizeInterior,
                           Building_SizeExterior,
                           Land_SizeTotal,
                           Land_SizeFrontage,
                           AlternateURL_DetailsLink,
                           RelativeDetailsURL,
                           AlternateURL_VideoLink,
                           PostalCode,
                           PublicRemarks,
                           ComputedSQFT,
                           ComputedPricePerSQFT,
                           ComputedLastUpdated
                      FROM Listings
                     WHERE 
                           Property_PriceUnformattedValue > {min_price} AND 
                           Property_PriceUnformattedValue < {max_price} AND 
                           {' AND '.join(conditions)};
                """
                rows = cursor.execute(query).fetchall()
                listings = [dict(x) for x in rows]
                logging.info(f"Received {len(listings)} listings from the DB")
                # specific_listing = [x for x in listings if x['MlsNumber'] == 26295500]
                # if specific_listing:
                #     pass
                if within_area_of_interest and self.area_of_interest:
                    listings = list(
                        filter(
                            lambda x: Polygon(self.area_of_interest).contains(
                                Point(x["Property_Address_Latitude"], x["Property_Address_Longitude"])
                            ),
                            listings,
                        )
                    )
                    logging.info(f"Filtered down to {len(listings)} listings because of area of interest")

                if min_metro_distance_meters:
                    listings = list(
                        filter(
                            lambda listing: any(
                                [
                                    distance.distance(
                                        [listing["Property_Address_Latitude"], listing["Property_Address_Longitude"]],
                                        poi,
                                    ).meters
                                    < min_metro_distance_meters
                                    for poi in self.points_of_interest
                                ]
                            ),
                            listings,
                        )
                    )
                    logging.info(f"Filtered down to {len(listings)} listings because of points of interest")

                if no_high_rise:
                    listings = list(
                        filter(
                            lambda x: not re.search(r"\|#([5-9]\d{2}|\d{4})\|", x["Property_Address_AddressText"]),
                            listings,
                        )
                    )
                    logging.info(f"Filtered down to {len(listings)} listings because of high apartments")

                return listings


    def export_data_to_csv(self, listings):
        listings_to_save = []
        cols_for_csv = [
            "MlsNumber",
            "Notes",
            "Property_PriceUnformattedValue",
            "ComputedPricePerSQFT",
            "Property_Address_AddressText",
            "Propery_CostPerSQFT",
            "Property_ParkingSpaceTotal",
            "Property_Parking",
            "Property_OwnershipType",
            "Property_Type",
            "Property_Photo_HighResPath",
            "Property_AmmenitiesNearBy",
            "ComputedSQFT",
            "InsertedDateUTC",
            "PriceChangeDateUTC",
            "Building_StoriesTotal",
            "Building_BathroomTotal",
            "Building_Bedrooms",
            "Building_Type",
            "Building_UnitTotal",
            "Building_SizeInterior",
            "Building_SizeExterior",
            "Land_SizeTotal",
            "Land_SizeFrontage",
            "AlternateURL_DetailsLink",
            "RelativeDetailsURL",
            "AlternateURL_VideoLink",
            "PublicRemarks",
            "ComputedLastUpdated",
        ]
        for listing in listings:
            has_custom_notes = listing["MlsNumber"] in self.mls_notes
            custom_notes = self.mls_notes[listing["MlsNumber"]].get("notes") if has_custom_notes else ""
            if custom_notes:
                listing["Notes"] = custom_notes
                if self.mls_notes.get(listing["MlsNumber"], {}).get("keep") is False:
                    continue
            listings_to_save.append(listing)
        with open("listings_to_audit.csv", "w") as csv_file:
            dict_writer = csv.DictWriter(csv_file, cols_for_csv, extrasaction='ignore')
            dict_writer.writeheader()
            dict_writer.writerows(listings_to_save)

    def display_listings_on_map(self, listings):
        pass
        # # Layers
        # folium.TileLayer("OpenStreetMap").add_to(my_map)
        # if os.environ["THUNDERFOREST_API_KEY"]:
        #     folium.TileLayer(
        #         "https://tile.thunderforest.com/transport/{z}/{x}/{y}.png?apikey="
        #         + os.environ["THUNDERFOREST_API_KEY"],
        #         name="Thunderforest Transportation",
        #         attr='&copy; <a href="http://www.thunderforest.com/">Thunderforest</a>, &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
        #     ).add_to(my_map)

# Area of Interest
We can set an area of interest to filter out listings.
For analysis, we likely won't want to use this.

In [3]:
aoi = [
    (45.546780742201165, -73.65807533729821),
    (45.5241750187359, -73.67472649086267),
    (45.51022227302072, -73.69086266029626),
    (45.50156020795671, -73.67524147499353),
    (45.48796289057615, -73.65258217323571),
    (45.467741340888665, -73.61258507240564),
    (45.45690538269222, -73.59181404579431),
    (45.454256276138466, -73.563661579974),
    (45.46990828260759, -73.55662346351892),
    (45.48038065986003, -73.54512215126306),
    (45.50601171342892, -73.5449504898861),
    (45.53241273092978, -73.54306221473962),
    (45.56006337665252, -73.6131000565365),
    (45.547682377783296, -73.63163948524743),
    (45.54972603156036, -73.65429878700525),
]

# Selecting Properties
There are a lot of properties available for sale. We'll want to filter out a lot of them.
Filtering is handle by both SQL and Python code.
The most costly option here is likely to be `min_metro_distance_meters` which checks each property against all possible metro stations.

In [4]:
db_file = "montreal.sqlite"
if not Path(db_file).exists():
    url = "https://github.com/alexmi256/property-analysis/releases/download/v0.0.1/montreal.tar.xz"
    file_name = "montreal.tar.xz"
    path, headers = urlretrieve(url, file_name)
    # print(f'Downloaded file {path}:\n{headers}')
    with tarfile.open(file_name) as f:
        f.extractall(filter="data")

In [5]:
viewer = MapViewer(db_file, area_of_interest=aoi)

relevant_listings = viewer.get_listings_from_db(
    min_price=400000,
    max_price=700000,
    within_area_of_interest=True,
    min_metro_distance_meters=1200,
    min_bedroom=2,
    min_sqft=900,
    max_price_per_sqft=700,
)

INFO:root:Received 4958 listings from the DB
INFO:root:Filtered down to 311 listings because of area of interest
INFO:root:Filtered down to 278 listings because of points of interest
INFO:root:Filtered down to 266 listings because of high apartments


In [6]:
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True

map_layout = Layout(height='1024px')

my_map = Map(layers=[mapnik], center=(45.5037, -73.6254), zoom=14, layout=map_layout)
# TODO: Add transit map if API key is available
my_map.add(LayersControl(position='topright'))
my_map.add(FullScreenControl())

if viewer.area_of_interest:
    polygon = Polyline(locations=viewer.area_of_interest, color='blue', fill=False)
    my_map.add(polygon)

for listing in relevant_listings:
    icon_color = "blue"
    marker_color = "white"
    has_custom_notes = listing["MlsNumber"] in viewer.mls_notes
    custom_notes = viewer.mls_notes[listing["MlsNumber"]].get("notes") if has_custom_notes else ""
    internet_status = "📠" if custom_notes and "bad_internet" in custom_notes.lower() else ""
    last_updated = '👴' if datetime.strptime(listing["ComputedLastUpdated"],
                                            '%Y-%m-%d') > datetime.now() + timedelta(days=-7) else '👶'

    if not listing["Property_Parking"]:
        garage_status = "❓🅿️"
    elif "Garage" in listing["Property_Parking"]:
        garage_status = "🅿️"
    else:
        garage_status = "🤔🅿️"

    if listing["PriceChangeDateUTC"]:
        price_history = "🗠" + listing["PriceChangeDateUTC"][:10]
        # TODO: Once we have a history db, try to look it up
    else:
        price_history = ""

    if listing['ComputedPricePerSQFT']:
        icon_color = MapViewer.get_color_for_number_between(listing['ComputedPricePerSQFT'])

    tooltip = f"${listing['Property_PriceUnformattedValue']}, {listing['Building_Bedrooms']}BDR ${listing['ComputedPricePerSQFT']}/sqft, {garage_status}{internet_status}{last_updated} {price_history} {custom_notes}"

    popup_html = f"""
                <img src="{listing['Property_Photo_HighResPath']}" width="320">
                <b>${listing['Property_PriceUnformattedValue']}</b> ${listing['ComputedPricePerSQFT']}/sqft {listing['MlsNumber']} {price_history}<br>
                {listing['Property_Address_AddressText']} <br>
                {listing['Building_Bedrooms']}BDR, {listing['Building_BathroomTotal']}BA, {listing['ComputedSQFT']}sqft, {listing['Building_Type']} <br>
                <a href="{listing['AlternateURL_DetailsLink']}" target="_blank">Details</a> <a href="https://www.realtor.ca{listing['RelativeDetailsURL']}" target="_blank">MLS</a> <br>
                Last Seen: {listing['ComputedLastUpdated']} <br>
                Parking: {listing['Property_Parking']}, {listing['Property_AmmenitiesNearBy']} <br>
                {custom_notes}
                """

    if has_custom_notes:
        if viewer.mls_notes[listing["MlsNumber"]].get("keep") is False:
            house_icon = "times-circle"
            marker_color = "lightgray"
        else:
            house_icon = "check-circle"
            marker_color = "lightblue"

    elif listing["Building_Type"] == "House":
        house_icon = "home"
    elif listing["Building_Type"] in ["Apartment"]:
        house_icon = "building"
    else:
        house_icon = "city"

    icon = AwesomeIcon(
        name=house_icon,
        icon_color=icon_color,
        marker_color=marker_color
    )

    marker = Marker(
        location=[listing["Property_Address_Latitude"], listing["Property_Address_Longitude"]],
        icon=icon,
        title=tooltip,
        draggable=False
    )
    marker.popup = HTML(popup_html)
    my_map.add(marker)

display(my_map)

Map(center=[45.5037, -73.6254], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# Heatmaps
Heatmaps using weights unfortunately don't work as expected in both leaflet and ploty
As far as I can tell, heatmaps only work based on density and not actual weights. 
You can test dis out by display clusters with different weights.

I have yet to try matplotlib but maybe that works better.

From reading docs, heatmaps weights should be between 0 and 1.0 by default but that doesn't matter.

## Heatmap Options
There are multiple parameters when we want to generate a heatmap:
- do we want it to be for the sample we're intrested in (i.e. price, features) or for a more general population
- do we want it to be based on listing price or price/sqft

The above doesn't matter if heatmaps don't work as expected however

In [7]:
# heatmap_listings = viewer.get_listings_from_db(
#     min_price=100000,
#     max_price=5000000,
#     must_have_int_sqft=False,
#     within_area_of_interest=True,
# )
# listings_price_sum = norm = sum(x['Property_PriceUnformattedValue'] for x in heatmap_listings)
# heat_data = [[float(x["Property_Address_Latitude"]), float(x["Property_Address_Longitude"]), float(x["Property_PriceUnformattedValue"])/listings_price_sum] for x in heatmap_listings]
# heatmap = Heatmap(locations=heat_data)
# my_map.add(heatmap)

# display(my_map)